In [1]:
import numpy as np
import time
from scipy.optimize import differential_evolution, minimize, dual_annealing
import matplotlib.pyplot as plt
import concurrent.futures
import threading, multiprocessing

import sys 
sys.path.append("../")  
import minionpy as mpy
import minionpy.test_functions as mpytest
import time
import iminuit

# Minimizing CEC benchmark problems with noise


In [2]:
# Global variables
N = 0  # Global counter for function evaluations
noise_ratio = 1e-4  # Noise level for the objective function
N_dict = {}  # Stores the number of function evaluations per algorithm
algos = ["ARRDE", "NelderMead", "L_BFGS_B", "DA"] #here, DA is dual annealing, ABC=artificial bee colony

def test_optimization_noise(func_number, year, bounds, dimension, func_name, Nmaxeval, seed):
    """
    Runs multiple optimization algorithms on the given function and stores results.
    
    Parameters:
    - func: Objective function to be minimized
    - bounds: Tuple representing the search space bounds
    - dimension: Number of dimensions for the problem
    - func_name: Name of the function (used for logging results)
    - Nmaxeval: Maximum number of function evaluations
    - seed: Random seed for reproducibility
    """
    global results, N, N_dict, noise_ratio, results_lock, algos

    if year == 2014:
        cec_func = mpy.CEC2014Functions(function_number=func_number, dimension=dimension)
    elif year == 2017:
        cec_func = mpy.CEC2017Functions(function_number=func_number, dimension=dimension)
    elif year == 2019:
        cec_func = mpy.CEC2019Functions(function_number=func_number)
    elif year == 2020:
        cec_func = mpy.CEC2020Functions(function_number=func_number, dimension=dimension)
    elif year == 2022:
        cec_func = mpy.CEC2022Functions(function_number=func_number, dimension=dimension)
    else:
        raise Exception("Unknown CEC year.")
    
    func = cec_func
    result = {}
    result['Dimensions'] = dimension
    result['Function'] = func_name
    bounds_list = [bounds] * dimension  # Extend bounds to all dimensions
    x0 = [[0.0 for _ in range(dimension)]]  # Initial starting point

    def func_wrapper(X):
        """Wraps the function to add evaluation tracking and noise."""
        global N
        ret = np.array(func(X))  # Compute function value
        N += len(X)  # Track the number of function evaluations
        return ret + noise_ratio * np.random.normal(size=len(X)) * np.abs(ret)  # Add noise
    
    def func_scipy(par):
        """Wrapper for compatibility with SciPy optimization methods."""
        return func_wrapper([par])[0]

    N = 0  # Reset evaluation counter
    res = mpy.Minimizer(
        func_wrapper, bounds_list, x0=x0, relTol=0.0, algo="L_BFGS_B", maxevals=Nmaxeval, 
        callback=None, seed=seed, options={
            "population_size": 0, 
            "N_points_derivative": 1,
            "use_local_search": True,
            "func_noise_ratio": noise_ratio
        }
    ).optimize()
    result["L_BFGS_B N=1"] = res.fun
    N_dict["L_BFGS_B N=1"] = N

    N = 0  # Reset evaluation counter
    res = mpy.Minimizer(
        func_wrapper, bounds_list, x0=x0, relTol=0.0, algo="L_BFGS_B", maxevals=Nmaxeval, 
        callback=None, seed=seed, options={
            "population_size": 0, 
            "N_points_derivative": 3,
            "use_local_search": True,
            "func_noise_ratio": noise_ratio
        }
    ).optimize()
    result["L_BFGS_B N=3"] = res.fun
    N_dict["L_BFGS_B N=3"] = N


    N = 0  # Reset evaluation counter
    res = mpy.Minimizer(
        func_wrapper, bounds_list, x0=x0, relTol=0.0, algo="L_BFGS_B", maxevals=Nmaxeval, 
        callback=None, seed=seed, options={
            "population_size": 0, 
            "N_points_derivative": 5,
            "use_local_search": True,
            "func_noise_ratio": noise_ratio
        }
    ).optimize()
    result["L_BFGS_B N=5"] = res.fun
    N_dict["L_BFGS_B N=5"] = N


    N = 0  # Reset evaluation counter
    res = mpy.Minimizer(
        func_wrapper, bounds_list, x0=x0, relTol=0.0, algo="L_BFGS_B", maxevals=Nmaxeval, 
        callback=None, seed=seed, options={
            "population_size": 0, 
            "N_points_derivative": 7,
            "use_local_search": True,
            "func_noise_ratio": noise_ratio
        }
    ).optimize()
    result["L_BFGS_B N=7"] = res.fun
    N_dict["L_BFGS_B N=7"] = N

    
    # Run L-BFGS-B from SciPy
    N = 0
    res_minimize = minimize(func_scipy, x0=x0[0], method="L-BFGS-B", options={"maxfun": Nmaxeval}, bounds=bounds_list)
    result["Scipy L_BFGS_B"] = res_minimize.fun
    N_dict["Scipy L_BFGS_B"] = N

    # Run Minuit Migrad from iminuit
    N = 0
    res_minimize = iminuit.minimize(func_scipy, x0=x0[0], method="migrad", options={"maxfun": Nmaxeval}, bounds=bounds_list)
    result["Minuit Migrad"] = res_minimize.fun
    N_dict["Minuit Migrad"] = N
    
    # Print results
    for res in result:
        if res == "Function" : 
            print("Function : ", result[res])
        if res not in ['Dimensions', 'Function']:
            print(f"\t{res:<20} : {result[res]:<10} \t N_evals : {N_dict[res]:<10}")
    print("")

def test_optimization(func, bounds, dimension, func_name, Nmaxeval, seed):
    """
    Runs multiple optimization algorithms on the given function and stores results.
    
    Parameters:
    - func: Objective function to be minimized
    - bounds: Tuple representing the search space bounds
    - dimension: Number of dimensions for the problem
    - func_name: Name of the function (used for logging results)
    - Nmaxeval: Maximum number of function evaluations
    - seed: Random seed for reproducibility
    """
    global results, N, N_dict, noise_ratio, results_lock, algos
    
    result = {}
    result['Dimensions'] = dimension
    result['Function'] = func_name
    bounds_list = [bounds] * dimension  # Extend bounds to all dimensions
    x0 = [[0.0 for _ in range(dimension)] ]# Initial starting point

    def func_wrapper(X):
        """Wraps the function to add evaluation tracking and noise."""
        global N
        ret = np.array(func(X))  # Compute function value
        N += len(X)  # Track the number of function evaluations
        return ret + noise_ratio * np.random.normal(size=len(X)) * np.abs(ret)  # Add noise
    
    def func_scipy(par):
        """Wrapper for compatibility with SciPy optimization methods."""
        return func_wrapper([par])[0]
    
    # Run various optimization algorithms
    for algo in algos:
        N = 0  # Reset evaluation counter
        res = mpy.Minimizer(
            func_wrapper, bounds_list, x0=x0, relTol=0.0, algo=algo, maxevals=Nmaxeval, 
            callback=None, seed=seed, options={
                "population_size": 0, 
                "N_points_derivative": 3,
                "use_local_search": True,
                "func_noise_ratio": noise_ratio
            }
        ).optimize()
        result[algo] = res.fun
        N_dict[algo] = N

    # Run L-BFGS method from minionpy
    N = 0
    res = mpy.L_BFGS(
        func_wrapper, x0=x0, relTol=0.0, maxevals=Nmaxeval, 
        callback=None, seed=seed, options={
            "population_size": 0, 
            "N_points_derivative": 3,
            "use_local_search": True,
            "func_noise_ratio": noise_ratio
        }
    ).optimize()
    result["L_BFGS"] = res.fun
    N_dict["L_BFGS"] = N
    
    # Run L-BFGS-B from SciPy
    N = 0
    res_minimize = minimize(func_scipy, x0=x0[0], method="L-BFGS-B", options={"maxfun": Nmaxeval}, bounds=bounds_list)
    result["Scipy L_BFGS_B"] = res_minimize.fun
    N_dict["Scipy L_BFGS_B"] = N

    # Run Dual Annealing from SciPy
    N = 0
    dual_ann = dual_annealing(func_scipy, bounds_list, x0=x0[0],maxfun=Nmaxeval, no_local_search=False)
    result["Scipy DA"] = dual_ann.fun
    N_dict["Scipy DA"] = N


    # Run Nelder-Mead from SciPy
    N = 0
    res_minimize = minimize(func_scipy, x0=x0[0], method="Nelder-Mead", options={"maxfev": Nmaxeval, "adaptive": True}, bounds=bounds_list)
    result["Scipy NelderMead"] = res_minimize.fun
    N_dict["Scipy NelderMead"] = N
  
    # Run Minuit Migrad from iminuit
    N = 0
    res_minimize = iminuit.minimize(func_scipy, x0=x0[0], method="migrad", options={"maxfun": Nmaxeval}, bounds=bounds_list)
    result["Minuit Migrad"] = res_minimize.fun
    N_dict["Minuit Migrad"] = N

    # Print results
    for res in result:
        if res == "Function" : 
            print("Function : ", result[res])
        if res not in ['Dimensions', 'Function']:
            print(f"\t{res:<20} : {result[res]:<10} \t N_evals : {N_dict[res]:<10}")
    print("")

def run_test_optimization(j, dim, year=2017, seed=None, Nmaxeval=10000):
    """
    Runs optimization tests for a specified CEC benchmark function.
    
    Parameters:
    - j: Function index in the CEC benchmark set
    - dim: Dimensionality of the function
    - year: Year of the CEC benchmark suite (default: 2017)
    - seed: Random seed for reproducibility
    - Nmaxeval: Maximum number of function evaluations
    """
    if year == 2014:
        cec_func = mpy.CEC2014Functions(function_number=j, dimension=dim)
    elif year == 2017:
        cec_func = mpy.CEC2017Functions(function_number=j, dimension=dim)
    elif year == 2019:
        cec_func = mpy.CEC2019Functions(function_number=j)
    elif year == 2020:
        cec_func = mpy.CEC2020Functions(function_number=j, dimension=dim)
    elif year == 2022:
        cec_func = mpy.CEC2022Functions(function_number=j, dimension=dim)
    else:
        raise Exception("Unknown CEC year.")
    
    test_optimization(cec_func, (-100, 100), dim, "func_" + str(j), Nmaxeval, seed)


# Performance of Minion's L-BFGS-B with Different $N$ Derivative Points

Minion's L-BFGS-B is vectorized and designed to be robust against noise. Function evaluations and their derivatives are computed in batches, ensuring efficient execution. To estimate derivatives, Minion employs the noise-robust Lanczos derivative method.

In the L-BFGS-B and L-BFGS settings, the key parameter `'N_points_derivative'` determines the number of points used for derivative calculation. This notebook compares the performance of L-BFGS-B with different values of `'N_points_derivative'`. A noise level of $10^{-4}$ is added to the CEC2017 benchmark problems to simulate real-world conditions.

- When $N = 1$, the numerical derivative reduces to the standard forward difference method.
- For $N \geq 2$, the Lanczos derivative formula is used.
- Specifically, $N = 3$ corresponds to the central difference method.

The following sections analyze how different values of `'N_points_derivative'` impact optimization performance.

In [3]:
# Counter for function evaluations
N = 0

# Noise ratio for function evaluations (set to zero for noiseless optimization)
noise_ratio = 1e-4

# Dictionary to store the number of evaluations per algorithm
N_dict = {}

# Maximum number of function evaluations allowed per optimization run
Nmaxeval = 100000  

# Dimensionality of the optimization problem
dimension = 10 

# Number of times each function should be tested (repetitions)
NRuns = 1  

# The CEC benchmark year to use for function selection

year = 2017  

# Dictionary mapping CEC benchmark years to their respective function sets
func_numbers_dict = {
    2022: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
    2020: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
    2019: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
    2017: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 
    2014: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
}

# Retrieve the function numbers for the selected benchmark year
func_numbers = func_numbers_dict[year]

for j in func_numbers:
    test_optimization_noise(j, year, (-100, 100), dimension, "func_"+str(j), Nmaxeval, None)

Function :  func_1
	L_BFGS_B N=1         : 155440809.72642672 	 N_evals : 2519      
	L_BFGS_B N=3         : 7795.616161828173 	 N_evals : 3234      
	L_BFGS_B N=5         : 4202.942475048814 	 N_evals : 3485      
	L_BFGS_B N=7         : 3646.845485023124 	 N_evals : 8235      
	Scipy L_BFGS_B       : 29974922011.342438 	 N_evals : 231       
	Minuit Migrad        : 29971490671.017815 	 N_evals : 3031      

Function :  func_2
	L_BFGS_B N=1         : 23408.84444166098 	 N_evals : 2618      
	L_BFGS_B N=3         : 1697216202721.2932 	 N_evals : 2877      
	L_BFGS_B N=5         : 1428.0826101061891 	 N_evals : 10086     
	L_BFGS_B N=7         : 1546.2404073861996 	 N_evals : 14335     
	Scipy L_BFGS_B       : 8.870509186758547e+17 	 N_evals : 231       
	Minuit Migrad        : 2.2047323275094912e+17 	 N_evals : 3203      

Function :  func_3
	L_BFGS_B N=1         : 4597.56779217251 	 N_evals : 4521      
	L_BFGS_B N=3         : 621.7111063178112 	 N_evals : 4410      
	L_BFGS_B N=5    

We observe that with $N=1$, where $N$ is the number of points used in the derivative calculation, L-BFGS-B is more robust than both SciPy's L-BFGS-B and Minuit Migrad. Notably, Minuit Migrad is a variant of the BFGS algorithm that has been widely used in high-energy physics for over 40 years due to its robustness. Our results confirm that Minuit Migrad is generally more robust than SciPy's L-BFGS-B.

When using higher values of $N$, we see that $N=3$ generally improves robustness but also requires more function evaluations. Increasing $N$ further, such as to $N=7$, does not provide significant improvements compared to using $N=3$ or $N=5$. Based on these findings, we recommend using either $N=3$ or $N=5$ as a trade-off between robustness and computational cost.

The number of function evaluations required for computing the function and its derivative, given $N$, is given by:

$$
\text{Function Calls} = 1 + D \cdot (N - 1)
$$

where $D$ is the dimensionality of the problem. These function evaluations are performed in batches to enhance efficiency.

What Happens if the Function is Smooth?
To analyze performance in a noise-free setting, we can compare results by setting the noise level to zero.

In [4]:
# Counter for function evaluations
N = 0

# Noise ratio for function evaluations (set to zero for noiseless optimization)
noise_ratio = 0.0

# Dictionary to store the number of evaluations per algorithm
N_dict = {}

# Maximum number of function evaluations allowed per optimization run
Nmaxeval = 100000  

# Dimensionality of the optimization problem
dimension = 10 

# Number of times each function should be tested (repetitions)
NRuns = 1  

# The CEC benchmark year to use for function selection

year = 2017  

# Dictionary mapping CEC benchmark years to their respective function sets
func_numbers_dict = {
    2022: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
    2020: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
    2019: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
    2017: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 
    2014: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
}

# Retrieve the function numbers for the selected benchmark year
func_numbers = func_numbers_dict[year]

for j in func_numbers:
    test_optimization_noise(j, year, (-100, 100), dimension, "func_"+str(j), Nmaxeval, None)

Function :  func_1
	L_BFGS_B N=1         : 100.02404359824438 	 N_evals : 1144      
	L_BFGS_B N=3         : 100.0      	 N_evals : 924       
	L_BFGS_B N=5         : 100.0      	 N_evals : 1763      
	L_BFGS_B N=7         : 100.0      	 N_evals : 2623      
	Scipy L_BFGS_B       : 100.00000379608935 	 N_evals : 341       
	Minuit Migrad        : 100.00000007939512 	 N_evals : 805       

Function :  func_2
	L_BFGS_B N=1         : 200.09360056634537 	 N_evals : 1958      
	L_BFGS_B N=3         : 200.00000912847912 	 N_evals : 3633      
	L_BFGS_B N=5         : 200.00334332154017 	 N_evals : 7093      
	L_BFGS_B N=7         : 200.000242639945 	 N_evals : 13786     
	Scipy L_BFGS_B       : 200.00004652119523 	 N_evals : 1485      
	Minuit Migrad        : 200.00024257079139 	 N_evals : 9267      

Function :  func_3
	L_BFGS_B N=1         : 300.0000000000427 	 N_evals : 627       
	L_BFGS_B N=3         : 300.00000000000296 	 N_evals : 1197      
	L_BFGS_B N=5         : 300.0000000000003 	 

We can see that using higher $N$ does not improves the performance. Therefore, for smooth function, $N=1$ can be safely used. 

# Performance of Minion's L-BFGS-B at Different Noise Levels

In this section, we compare the performance of various algorithms implemented in Minion against their counterparts in other libraries, such as SciPy and Minuit. The test function used is the CEC2017 benchmark function with a dimensionality of $D = 10$.

## Noise Level : 0.01

In [5]:
# Counter for function evaluations
N = 0

# Noise ratio for function evaluations (set to zero for noiseless optimization)
noise_ratio = 0.01

# Dictionary to store the number of evaluations per algorithm
N_dict = {}

# Maximum number of function evaluations allowed per optimization run
Nmaxeval = 100000  

# Dimensionality of the optimization problem
dimension = 10 

# Number of times each function should be tested (repetitions)
NRuns = 1  

# The CEC benchmark year to use for function selection
year = 2017  

# Dictionary mapping CEC benchmark years to their respective function sets
func_numbers_dict = {
    2022: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
    2020: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
    2019: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
    2017: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 
    2014: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
}

# Retrieve the function numbers for the selected benchmark year
func_numbers = func_numbers_dict[year]

# Using a thread pool to execute optimization tasks in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    futures = []  # List to store future objects representing scheduled tasks

    # Run optimization tests multiple times (for averaging results)
    for k in range(NRuns):
        for j in func_numbers:
            # Submit the optimization test function to the thread pool
            futures.append(executor.submit(run_test_optimization, j, dimension, year, k, Nmaxeval))

    # Wait for all submitted tasks to complete
    concurrent.futures.wait(futures)

    # Retrieve and process results (ensure all threads completed successfully)
    for f in futures:
        f.result()

Function :  func_1
	ARRDE                : 96.83056355792034 	 N_evals : 100045    
	NelderMead           : 9817.135238543873 	 N_evals : 100002    
	L_BFGS_B             : 28731749346.948776 	 N_evals : 1827      
	DA                   : 7897.666034163657 	 N_evals : 100016    
	L_BFGS               : 28960575391.590675 	 N_evals : 3004      
	Scipy L_BFGS_B       : 29753789678.50404 	 N_evals : 330       
	Scipy DA             : 3831.710700738001 	 N_evals : 46643     
	Scipy NelderMead     : 28602754549.99285 	 N_evals : 100000    
	Minuit Migrad        : 29449489984.593414 	 N_evals : 1773      

Function :  func_2
	ARRDE                : 190.81799856352387 	 N_evals : 100045    
	NelderMead           : 1854199.683945974 	 N_evals : 100009    
	L_BFGS_B             : 1603191813016.7212 	 N_evals : 2583      
	DA                   : 227.3965489355769 	 N_evals : 100019    
	L_BFGS               : 8.557274387291581e+17 	 N_evals : 1366      
	Scipy L_BFGS_B       : 8.787431660279418e

## Noise Level: 0.0001 

In [6]:
# Noise ratio for function evaluations (set to zero for noiseless optimization)
noise_ratio = 1e-4

# Using a thread pool to execute optimization tasks in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    futures = []  # List to store future objects representing scheduled tasks

    # Run optimization tests multiple times (for averaging results)
    for k in range(NRuns):
        for j in func_numbers:
            # Submit the optimization test function to the thread pool
            futures.append(executor.submit(run_test_optimization, j, dimension, year, k, Nmaxeval))

    # Wait for all submitted tasks to complete
    concurrent.futures.wait(futures)

    # Retrieve and process results (ensure all threads completed successfully)
    for f in futures:
        f.result()

Function :  func_1
	ARRDE                : 99.97398954921276 	 N_evals : 100045    
	NelderMead           : 250425718.5703963 	 N_evals : 100003    
	L_BFGS_B             : 2878.6106795768783 	 N_evals : 3108      
	DA                   : 9035.479444262133 	 N_evals : 100011    
	L_BFGS               : 1836.2246069441626 	 N_evals : 2416      
	Scipy L_BFGS_B       : 29976884575.383377 	 N_evals : 418       
	Scipy DA             : 662.9100090171434 	 N_evals : 53375     
	Scipy NelderMead     : 29963325809.53629 	 N_evals : 100000    
	Minuit Migrad        : 29972756324.34399 	 N_evals : 3175      

Function :  func_2
	ARRDE                : 199.92291590576446 	 N_evals : 100044    
	NelderMead           : 1540.4934804779364 	 N_evals : 100006    
	L_BFGS_B             : 7.502755353132613e+17 	 N_evals : 1386      
	DA                   : 199.92597744190107 	 N_evals : 100012    
	L_BFGS               : 7.564475030591148e+17 	 N_evals : 1366      
	Scipy L_BFGS_B       : 8.86969616469

## Noise Level : 1e-6

In [7]:
# Noise ratio for function evaluations (set to zero for noiseless optimization)
noise_ratio = 1e-6

# Using a thread pool to execute optimization tasks in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    futures = []  # List to store future objects representing scheduled tasks

    # Run optimization tests multiple times (for averaging results)
    for k in range(1):
        for j in func_numbers:
            # Submit the optimization test function to the thread pool
            futures.append(executor.submit(run_test_optimization, j, dimension, year, k, Nmaxeval))

    # Wait for all submitted tasks to complete
    concurrent.futures.wait(futures)

    # Retrieve and process results (ensure all threads completed successfully)
    for f in futures:
        f.result()

Function :  func_1
	ARRDE                : 99.99965568441243 	 N_evals : 100045    
	NelderMead           : 411510483.0143577 	 N_evals : 100008    
	L_BFGS_B             : 99.99974266892346 	 N_evals : 3255      
	DA                   : 99.99964758925897 	 N_evals : 100021    
	L_BFGS               : 102.51224266242104 	 N_evals : 2584      
	Scipy L_BFGS_B       : 29975433231.454494 	 N_evals : 341       
	Scipy DA             : 9525.140513432096 	 N_evals : 55883     
	Scipy NelderMead     : 3261974710.6240864 	 N_evals : 100000    
	Minuit Migrad        : 4696.276270522521 	 N_evals : 8204      

Function :  func_2
	ARRDE                : 199.9994039717403 	 N_evals : 100045    
	NelderMead           : 280.6718174861575 	 N_evals : 100012    
	L_BFGS_B             : 200.0015072927467 	 N_evals : 3696      
	DA                   : 199.99983015965012 	 N_evals : 100017    
	L_BFGS               : 7.556722049752932e+17 	 N_evals : 1366      
	Scipy L_BFGS_B       : 8.869638406639756e+

## Noise Level : 1e-8

In [8]:
# Noise ratio for function evaluations (set to zero for noiseless optimization)
noise_ratio = 1e-8

# Using a thread pool to execute optimization tasks in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    futures = []  # List to store future objects representing scheduled tasks

    # Run optimization tests multiple times (for averaging results)
    for k in range(NRuns):
        for j in func_numbers:
            # Submit the optimization test function to the thread pool
            futures.append(executor.submit(run_test_optimization, j, dimension, year, k, Nmaxeval))

    # Wait for all submitted tasks to complete
    concurrent.futures.wait(futures)

    # Retrieve and process results (ensure all threads completed successfully)
    for f in futures:
        f.result()

Function :  func_1
	ARRDE                : 99.99999751719136 	 N_evals : 100045    
	NelderMead           : 411510501.72534627 	 N_evals : 100006    
	L_BFGS_B             : 99.99999779301376 	 N_evals : 1869      
	DA                   : 99.99999583687155 	 N_evals : 100008    
	L_BFGS               : 99.99999811741898 	 N_evals : 2290      
	Scipy L_BFGS_B       : 22097418029.7505 	 N_evals : 836       
	Scipy DA             : 5176.843214574206 	 N_evals : 40747     
	Scipy NelderMead     : 2422.988133120459 	 N_evals : 6951      
	Minuit Migrad        : 100.00006918869452 	 N_evals : 816       

Function :  func_2
	ARRDE                : 199.99999339314 	 N_evals : 100045    
	NelderMead           : 340.031347288827 	 N_evals : 100001    
	L_BFGS_B             : 200.00084391445952 	 N_evals : 4935      
	DA                   : 200.00050007089212 	 N_evals : 100015    
	L_BFGS               : 7.556704944803493e+17 	 N_evals : 1366      
	Scipy L_BFGS_B       : 8.869645460851034e+17 	

## Noise Level : 1e-10

In [9]:
# Noise ratio for function evaluations (set to zero for noiseless optimization)
noise_ratio = 1e-10

# Using a thread pool to execute optimization tasks in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    futures = []  # List to store future objects representing scheduled tasks

    # Run optimization tests multiple times (for averaging results)
    for k in range(NRuns):
        for j in func_numbers:
            # Submit the optimization test function to the thread pool
            futures.append(executor.submit(run_test_optimization, j, dimension, year, k, Nmaxeval))

    # Wait for all submitted tasks to complete
    concurrent.futures.wait(futures)

    # Retrieve and process results (ensure all threads completed successfully)
    for f in futures:
        f.result()

Function :  func_1
	ARRDE                : 99.99999998146482 	 N_evals : 100045    
	NelderMead           : 411510502.8785815 	 N_evals : 100008    
	L_BFGS_B             : 100.00000000471442 	 N_evals : 903       
	DA                   : 99.9999999675947 	 N_evals : 100017    
	L_BFGS               : 99.99999996461153 	 N_evals : 925       
	Scipy L_BFGS_B       : 100.02471298242172 	 N_evals : 1386      
	Scipy DA             : 100.00062903391391 	 N_evals : 21090     
	Scipy NelderMead     : 2712.3754020719502 	 N_evals : 6914      
	Minuit Migrad        : 100.00002479993728 	 N_evals : 746       

Function :  func_2
	ARRDE                : 199.99999998141658 	 N_evals : 100045    
	NelderMead           : 295.94632102454506 	 N_evals : 100004    
	L_BFGS_B             : 200.00015248095772 	 N_evals : 4809      
	DA                   : 200.00005569235267 	 N_evals : 100004    
	L_BFGS               : 7.556706336095375e+17 	 N_evals : 1366      
	Scipy L_BFGS_B       : 301.15054725233

We can that L-BFGS-B are generally more robust than similar algorithms implemented by Scipy and Minuit library.